# Bulk procesor

In [1]:
from collections import namedtuple
from copy import deepcopy

from ssscoring import BREAKOFF_ALTITUDE
from ssscoring import PERFORMANCE_WINDOW_LENGTH
from ssscoring import aggregateResults
from ssscoring import convertFlySight2SSScoring
from ssscoring import dropNonSkydiveDataFrom
from ssscoring import getAllSpeedJumpFilesFrom
from ssscoring import getSpeedSkydiveFrom
from ssscoring import isValidJump
from ssscoring import jumpAnalysisTable
from ssscoring import processAllJumpFiles
from ssscoring import roundedAggregateResults
from ssscoring import totalResultsFrom
from ssscoring import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import ipywidgets as widgets
import pandas as pd

In [2]:
DATA_LAKE_ROOT = './data'

In [3]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [4]:
jumpResults = processAllJumpFiles(jumpFiles)

---
## Results

In [5]:
aggregate = aggregateResults(jumpResults)
sum = totalResultsFrom(aggregate)
display(aggregate)
display(sum)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
10-22-26,436.95,140.490585,107.614701,98.944747,80.64875,58.3924,25.0,440.316


,totalSpeed,meanSpeed,maxScore
totalSpeed,436.95,436.95,436.95


### Rounded results for training log

In [6]:
roundedAggregateResults(jumpResults)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
10-22-26,437,140,108,99,81,58,25.0,440


In [7]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in jumpResults.keys():
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)</h3>'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        graphJumpResult(initializePlot(resultRef),
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">10-22-26 jump - 🟢 valid</span></h1>')

HTML(value='<h3>Max speed = 440; exit at 4031 m (13227 ft), end scoring window at 1775 m (5825 ft)</h3>')

## All jumps

In [8]:
bp.show(allJumpsPlot)